In [1]:
import re
import sys
import json

import pandas as pd
import numpy as np

import repetition_matrix

import seaborn as sns

In [2]:
from lyricsgenius import Genius

genius = Genius(
    'in424DYZRH1oa--Iyzce_Ue6uLYNqt1Q6-Yq20PUSYl-PjvMNuuaX0YBiwcSFt9_',
    retries=50
)

In [3]:
song_parts_regex = r"\[Intro.*?\]|\[Verse [0-9].*?\]|\[Refrain.*?\]|\[Pre-Chorus.*?\]|\[Chorus.*?\]|\[Post-Chorus.*?\]|\[Hooks.*?\]|\[Riffs/Basslines.*?\]|\[Scratches.*?\]|\[Bridge.*?\]|\[Interlude.*?\]|\[Break.*?\]|\[Skit.*?\]|\[Collision.*?\]|\[Instrumental or Solo.*?\]|\[Ad lib.*?\]|\[Segue.*?\]|\[Outro.*?\]"

In [22]:
def get_chorus(song_object):
    raw_lyrics_list = song_object['lyrics'].split('\n')
    filtered_lyrics_list = filter(lambda line: line != '', raw_lyrics_list)
    chorus = []
    chorus_indicator = False
    for line in filtered_lyrics_list:
        if chorus_indicator:
            if re.match(song_parts_regex, line):
                break
            chorus.append(line)
        if re.match(r"\[Chorus.*?\]", line):
            chorus_indicator = True
    return chorus

In [23]:
# def load_jsons():
#     song_objects = []
#     for i in range(0, 23500, 100):
#         with open(f'test_data/song_jsons_{i}_to_{i+100}.json') as json_file:
#             data = json.load(json_file)
#             song_objects += data
#     return song_objects

In [36]:
with open(f'data/all_songs.json') as json_file:
    song_objects = json.load(json_file)
    print(song_objects['0'])
    song_objects = [song_objects[index] for index in song_objects]

{'song': 'Blinding Lights by The Weeknd', 'artist': 'The Weeknd', 'chorus': ["I said, ooh, I'm blinded by the lights", "No, I can't sleep until I feel your touch", "I said, ooh, I'm drowning in the night", "Oh, when I'm like this, you're the one I trust", 'Hey, hey, hey']}


In [40]:
with open(f'data/all_songs.json', 'x') as json_file:
    json.dump(song_objects, json_file)

In [38]:
songs = list(filter(lambda song: len(song['chorus']) >= 4, song_objects))
songs

[{'song': 'Blinding Lights by The Weeknd',
  'artist': 'The Weeknd',
  'chorus': ["I said, ooh, I'm blinded by the lights",
   "No, I can't sleep until I feel your touch",
   "I said, ooh, I'm drowning in the night",
   "Oh, when I'm like this, you're the one I trust",
   'Hey, hey, hey']},
 {'song': 'Radioactive by Imagine Dragons',
  'artist': 'Imagine Dragons',
  'chorus': ['Whoa-oh, whoa',
   "I'm radioactive, radioactive",
   'Whoa-oh, whoa',
   "I'm radioactive, radioactive"]},
 {'song': 'Sail by AWOLNATION',
  'artist': 'AWOLNATION',
  'chorus': ['Sail', 'Sail', 'Sail', 'Sail', 'Sail']},
 {'song': 'How Do I Live by LeAnn Rimes',
  'artist': 'LeAnn Rimes',
  'chorus': ['How do I live without you? I want to know',
   'How do I breathe without you if you ever go?',
   'How do I ever, ever survive?',
   'How do I, how do I, oh, how do I live?']},
 {'song': 'Counting Stars by OneRepublic',
  'artist': 'OneRepublic',
  'chorus': ["Lately, I've been, I've been losin' sleep",
   "Dreami

In [41]:
df = pd.DataFrame(songs)

In [43]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

2022-12-05 13:51:21.535751: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-05 13:51:21.536057: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-12-05 13:51:24.429963: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-05 13:51:24.449831: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [60]:
def calculate_similarity_score(chorus):
    chorus_embeddings = embed(chorus)
    corr = np.inner(chorus_embeddings, chorus_embeddings).tolist()

    indices = range(len(corr))
    for row in indices:
        for col in indices:
            if corr[row][col] > 0.99:
                corr[row][col] = 1
            if corr[row][col] < 0:
                corr[row][col] = 0


    count_not_identity = len(corr) ** 2 - len(corr)
    avg_similarity = 0
    
    for row in indices:
        for col in indices:
            if row != col:
                avg_similarity += corr[row][col] / count_not_identity
    return avg_similarity

In [61]:
calculate_similarity_score(songs[0]['chorus'])

[[1.0000004768371582, 0.2504209578037262, 0.5864431858062744, 0.3111353814601898, 0.16396911442279816], [0.2504209578037262, 1.0, 0.43166258931159973, 0.2537192106246948, 0.06480016559362411], [0.5864431858062744, 0.43166258931159973, 0.9999998807907104, 0.3234749734401703, 0.17213916778564453], [0.3111353814601898, 0.2537192106246948, 0.3234749734401703, 0.9999997615814209, 0.0929884985089302], [0.16396911442279816, 0.06480016559362411, 0.17213916778564453, 0.0929884985089302, 0.9999997019767761]]


0.26507532447576526

In [62]:
songs[0]

{'song': 'Blinding Lights by The Weeknd',
 'artist': 'The Weeknd',
 'chorus': ["I said, ooh, I'm blinded by the lights",
  "No, I can't sleep until I feel your touch",
  "I said, ooh, I'm drowning in the night",
  "Oh, when I'm like this, you're the one I trust",
  'Hey, hey, hey']}